In [1]:
import os, json, difflib
from pprint import pprint
import pandas as pd

In [6]:
def group_description_according_to_module_and_version_then_print():
    raw_file_path = "J:/EMSE2020_MLPackage/Data/TFHub/2020-03-10/TFHubModule_Info.json"
    with open(raw_file_path, "r") as rfp:
        modules = json.load(rfp)

    module_names = {} ## {"module1": {"v1": "xxxxxx", "v2": "xxxxxx"}, ......}
    for module in modules:
        ## 跳过一些
        moduleFormat = module["info_NormalDeployment"][-1][-11]
        if moduleFormat not in ("saved_model_2", "hub", "saved_model"):
            continue
        ###
        name = module["name"]
        author = None
        for l in module["info_NormalDeployment"][-1][42]:
            if l[0] == "publisher":
                author = l[1][0][0]
                break
        name = "{}@@{}".format(name, author)
        version = module["version"]
        description = module["info_NormalDeployment"][1][7] ## this is how to get the html version: module["info"][2]
        if name not in module_names:
            module_names[name] = {version: description}
        else:
            module_names[name][version] = description
#     print(module_names)
    path_lv1 = "module_descriptions"
    if not os.path.exists(path_lv1):
        os.makedirs(path_lv1)
    for module in module_names:
        path_lv2 = os.path.join(path_lv1, module)
        if not os.path.exists(path_lv2):
            os.makedirs(path_lv2)
        ## 
        versions = module_names[module] 
#         print(versions)
#         with open("module_descriptions/{}.md".format(module), "w") as mdm:
        for vNum in versions:
            file_path = os.path.join(path_lv2, "{}.md".format(vNum))
            dscp = versions[vNum]
            with open(file_path, "w", encoding="utf-8") as fp:
                fp.write(dscp)
    return

group_description_according_to_module_and_version_then_print()

In [7]:
def go_through_the_files(keyword):
    raw_file_path = "J:/ModelStoreData/AIHub/2020-03-10/TFHubModule_Info.json"
    with open(raw_file_path, "r") as rfp:
        modules = json.load(rfp)

    module_names = {} ## {"module1": {"v1": "xxxxxx", "v2": "xxxxxx"}, ......}
    for module in modules:
        ## 跳过一些
        moduleFormat = module["info_NormalDeployment"][-1][-11]
        if moduleFormat not in ("saved_model_2", "hub", "saved_model"):
            continue
        ##
        name = module["name"]
        version = module["version"]
        author = None
        for l in module["info_NormalDeployment"][-1][42]:
            if l[0] == "publisher":
                author = l[1][0][0]
                break
        name = "{}@@{}".format(name, author)
        description = module["info_NormalDeployment"][1][7] ## this is how to get the html version: module["info"][2]
        if keyword in description:
            print(name, version) 
    
    return

go_through_the_files("heckpoint")
### The outputs are the 

imagenet-mobilenet_v1_075_192-quantops-feature_vector@@google 1
imagenet-mobilenet_v1_075_192-quantops-feature_vector@@google 3
imagenet-mobilenet_v1_050_160-classification@@google 1
imagenet-mobilenet_v1_050_160-classification@@google 3
imagenet-mobilenet_v1_050_160-classification@@google 4
imagenet-mobilenet_v1_050_192-quantops-feature_vector@@google 1
imagenet-inception_resnet_v2-classification@@google 1
imagenet-mobilenet_v1_050_192-quantops-feature_vector@@google 3
imagenet-inception_resnet_v2-classification@@google 3
imagenet-nasnet_large-classification@@google 1
imagenet-nasnet_large-classification@@google 3
imagenet-mobilenet_v2_050_96-feature_vector@@google 1
imagenet-mobilenet_v2_050_96-feature_vector@@google 2
imagenet-mobilenet_v2_050_96-feature_vector@@google 3
imagenet-mobilenet_v2_050_96-feature_vector@@google 4
imagenet-mobilenet_v2_075_96-feature_vector@@google 1
imagenet-nasnet_large-classification@@google 4
imagenet-mobilenet_v2_075_96-feature_vector@@google 2
imagen

The following methods: `compare_file1_and_file2(_xx)` are to generate the comparison results between neighbouring versions. 

In [8]:
def compare_file1_and_file2(file1_path, file1_name, file2_path, file2_name, rst_dir, module):
    def readfile(path):
        with open(path, "r", encoding="utf-8") as p:
            text = p.readlines()
        return text

    text1_lines = readfile(file1_path)
    text2_lines = readfile(file2_path)
    d = difflib.Differ() #.HtmlDiff()
    diff_rst = list(d.compare(text1_lines, text2_lines))
    with open(os.path.join(rst_dir, "{}.md".format(module)), "a") as rp:
        rp.write("\n## {} -> {}\n".format(file1_name, file2_name))
        for line in diff_rst:
            if line[0:2] == "+ " or line[0:2] == "- " or line[0:2] == "? ":
                rp.write(
                    "    {}".format(line) # if "heckpoint" in line
                    # else  "    {}".format(line) 
                )
                if line[0:2] == "? ":
                    rp.write("-----------------------------\n\n")
    return

# def compare_file1_and_file2_only_ckpt_info(file1_path, file1_name, file2_path, file2_name, rst_dir, module):
#     def readfile(path):
#         with open(path, "r", encoding="utf-8") as p:
#             text = p.readlines()
#         return text

#     text1_lines = readfile(file1_path)
#     text2_lines = readfile(file2_path)
#     d = difflib.Differ() #.HtmlDiff()
#     diff_rst = list(d.compare(text1_lines, text2_lines))
    
#     caption_written = False
#     for line in diff_rst:
#         if (line[0:2] == "+ " or line[0:2] == "- " or line[0:2] == "? ") and ("heckpoint" in line):
#             with open(os.path.join(rst_dir, "{}.md".format(module)), "a") as rp:
#                 if not caption_written: 
#                     rp.write("\n## {} -> {}\n".format(file1_name, file2_name))
#                     caption_written = True
#                 rp.write(
#                     "    {}".format(line)
#                 )                
#     return

def compare_file1_and_file2_compare_ckpt_related_lines(file1_path, file1_name, file2_path, file2_name, rst_dir, module, keywords):
    def readfile(path):
        with open(path, "r", encoding="utf-8") as p:
            text = p.readlines()
        return text

    text1_lines = readfile(file1_path)
    text2_lines = readfile(file2_path)
    d = difflib.Differ() #.HtmlDiff()
    diff_rst = list(d.compare(text1_lines, text2_lines))
    
    ckpt_related_lines = []
    for line in diff_rst:
        if (line[0:2] == "+ " or line[0:2] == "- " or line[0:2] == "? ") and (keywords in line): 
            line = line[2:].replace("module", "model")
            if line not in ckpt_related_lines:
                ckpt_related_lines.append(line)
            else:
                pass
    if len(ckpt_related_lines) > 1:
        print(module)
    return

def compare_files(dscp_path):
    for module in os.listdir(dscp_path):
        path_lv1 = os.path.join(dscp_path, module)
        file_list = os.listdir(path_lv1)
        for i in range(0, len(file_list) - 1):
#             compare_file1_and_file2(
#                 os.path.join(path_lv1, file_list[i]), 
#                 file_list[i], 
#                 os.path.join(path_lv1, file_list[i + 1]),
#                 file_list[i + 1], 
#                 r"J:\CrawlerForModelStore\MLTechTransfer\issue 18 Refresh the families\module_dscp_diff_rst", 
#                 module
#             )
            compare_file1_and_file2_compare_ckpt_related_lines( 
                os.path.join(path_lv1, file_list[i]), 
                file_list[i], 
                os.path.join(path_lv1, file_list[i + 1]),
                file_list[i + 1], 
                r"J:\CrawlerForModelStore\MLTechTransfer\issue 18 Refresh the families\module_dscp_diff_rst", 
                module, 
                "rain"
            )
#             print(file_list[i], "-->", file_list[i + 1])
compare_files(r"module_descriptions")

elmo@@google
elmo@@google
imagenet-inception_resnet_v2-classification@@google
imagenet-inception_resnet_v2-classification@@google
imagenet-inception_resnet_v2-feature_vector@@google
imagenet-inception_resnet_v2-feature_vector@@google
imagenet-inception_v1-classification@@google
imagenet-inception_v1-classification@@google
imagenet-inception_v1-feature_vector@@google
imagenet-inception_v1-feature_vector@@google
imagenet-inception_v2-classification@@google
imagenet-inception_v2-classification@@google
imagenet-inception_v2-feature_vector@@google
imagenet-inception_v2-feature_vector@@google
imagenet-inception_v3-classification@@google
imagenet-inception_v3-classification@@google
imagenet-inception_v3-feature_vector@@google
imagenet-inception_v3-feature_vector@@google
imagenet-mobilenet_v1_025_128-classification@@google
imagenet-mobilenet_v1_025_128-classification@@google
imagenet-mobilenet_v1_025_128-feature_vector@@google
imagenet-mobilenet_v1_025_128-feature_vector@@google
imagenet-mobil

imagenet-mobilenet_v2_075_96-classification@@google
imagenet-mobilenet_v2_075_96-classification@@google
imagenet-mobilenet_v2_075_96-classification@@google
imagenet-mobilenet_v2_075_96-feature_vector@@google
imagenet-mobilenet_v2_075_96-feature_vector@@google
imagenet-mobilenet_v2_075_96-feature_vector@@google
imagenet-mobilenet_v2_100_128-classification@@google
imagenet-mobilenet_v2_100_128-classification@@google
imagenet-mobilenet_v2_100_128-classification@@google
imagenet-mobilenet_v2_100_128-feature_vector@@google
imagenet-mobilenet_v2_100_128-feature_vector@@google
imagenet-mobilenet_v2_100_128-feature_vector@@google
imagenet-mobilenet_v2_100_160-classification@@google
imagenet-mobilenet_v2_100_160-classification@@google
imagenet-mobilenet_v2_100_160-classification@@google
imagenet-mobilenet_v2_100_160-feature_vector@@google
imagenet-mobilenet_v2_100_160-feature_vector@@google
imagenet-mobilenet_v2_100_160-feature_vector@@google
imagenet-mobilenet_v2_100_192-classification@@google